# Experiments for CS224U Project

## Setup

### Imports

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
import csv
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [2]:
import utils
import sst
import scipy.stats
from sgd_classifier import BasicSGDClassifier

In [3]:
from sklearn.linear_model import LogisticRegression
import os

In [6]:
import numpy as np

### Dataset

In [7]:
def read_array_from_csv(inputcsv):
    out = []
    with open(inputcsv, encoding='utf-8') as f:
        reader = csv.reader(f)
        for q in reader:
            out.append(q)
    return out

In [8]:
anon_new_train = read_array_from_csv('data/anon_train_dataUnbiased.csv')

### SST Machinery

In [9]:
def hansard_reader(
        src_filename,
        class_func=None):
    """Overview

    Parameters
    ----------
    src_filename : str
        Full path to the file to be read.
    class_func : None, or function mapping labels to labels or None
        If this is None, then the original 5-way labels are returned.
        Other options: `binary_class_func` and `ternary_class_func`
        (or you could write your own).


    Yields
    ------
    (tree, label)
        nltk.Tree, str in {'0','1','2','3','4'}

    """
    if class_func is None:
        class_func = lambda x: x
    with open(src_filename, encoding='utf-8') as f:
        reader = csv.reader(f)
        for q in reader:
            yield (q[0], class_func(q[1]))

We need a reader for each dataset, both for train and for test.

First, the standard data:

In [10]:
def train_reader(**kwargs):
    """Convenience function for reading the train file, full-trees only."""
    src = 'data/anon_train_dataUnbiased.csv'
    return hansard_reader(src,**kwargs)

Next, the anonymised data:

In [11]:
def anon_train_reader(**kwargs):
    src = 'data/anon_train_dataUnbiased.csv'
    return hansard_reader(src,**kwargs)

The test readers won't be used until the *very* end.

### Class Functions

In [12]:
def cas_to_gov(label):
    if label == 'cas':
        return 'gov'
    else:
        return label

## Baselines

### Bag-of-words Feature Functions

A unigrams feature function

In [13]:
def unigrams_phi(question):
    """The basis for a unigrams feature function.
    Parameters
    ----------
    question : string
        The question to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in the question. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    """
    unigrams = {}
    for word in question.split() :
        unigrams[word] = unigrams.get(word, 0) + 1
    return unigrams

A bigrams feature function

In [14]:
def bigrams_phi(question):
    """The basis for a unigrams feature function.
    
    Parameters
    ----------
    tree : nltk.tree
        The tree to represent.
    
    Returns
    -------    
    defaultdict
        A map from strings to their counts in `tree`. (Counter maps a 
        list to a dict of counts of the elements in that list.)
    
    """
    bigrams = {}
    qarray = question.split()
    for i in range(0, len(qarray)-1) :
        big = qarray[i] + '_' + qarray[i+1]
        bigrams[big] = bigrams.get(big, 0) + 1
    return bigrams

A basic bag-of-words unigrams and bigrams feature function

In [15]:
def uni_bigrams_phi(question):
    grams = unigrams_phi(question)
    grams.update(bigrams_phi(question))
    return grams

We found that *friend* seems to be a good indicator. What happens if we only give the classifier that feature? Or unigrams without it?

In [16]:
def only_friend_phi(question):
    if 'friend' in question.lower().split():
        return {'friend':1}
    else:
        return {'friend':0}

In [17]:
def no_friends_phi(question):
    unigrams = {}
    for word in question.split() :
        if word.lower() is not 'friend':
            unigrams[word.lower()] = unigrams.get(word.lower(), 0) + 1
    return unigrams

### SGD Classifier Baseline
Stochastic Gradient Descent

In [18]:
def fit_basic_sgd_classifier(X, y):    
    """Wrapper for `BasicSGDClassifier`.
    
    Parameters
    ----------
    X : 2d np.array
        The matrix of features, one example per row.        
    y : list
        The list of labels for rows in `X`.
    
    Returns
    -------
    BasicSGDClassifier
        A trained `BasicSGDClassifier` instance.
    
    """    
    mod = BasicSGDClassifier()
    mod.fit(X, y)
    return mod

#### Experiments

An SGD classifier trained on unigrams for the modified dataset.

In [19]:
_ = sst.experiment(
    unigrams_phi,
    fit_basic_sgd_classifier,
    train_reader=anon_train_reader, 
    assess_reader=None, 
    train_size=0.7,
    class_func=cas_to_gov,
    score_func=utils.safe_macro_f1,
    verbose=True)

Accuracy: 0.725
             precision    recall  f1-score   support

        gov      0.708     0.761     0.734      1057
        opp      0.743     0.689     0.715      1064

avg / total      0.726     0.725     0.724      2121



An SGD classifier trained on bigrams from the modified dataset.

In [20]:
_ = sst.experiment(
    bigrams_phi,
    fit_basic_sgd_classifier,
    train_reader=anon_train_reader, 
    assess_reader=None, 
    train_size=0.7,
    class_func=cas_to_gov,
    score_func=utils.safe_macro_f1,
    verbose=True)

Accuracy: 0.740
             precision    recall  f1-score   support

        gov      0.744     0.736     0.740      1058
    memAffi      0.000     0.000     0.000         0
        opp      0.742     0.743     0.742      1063

avg / total      0.743     0.740     0.741      2121



### Logistic Regression Baseline

In [21]:
def fit_maxent_classifier(X, y):   
    mod = LogisticRegression(fit_intercept=True)
    mod.fit(X, y)
    return mod

#### Experiments

Again, we start with unigrams for the basic dataset.

Now unigrams for the modified dataset.

In [22]:
_ = sst.experiment(
    unigrams_phi,
    fit_maxent_classifier,
    train_reader=anon_train_reader, 
    assess_reader=None, 
    train_size=0.7,
    class_func=cas_to_gov,
    score_func=utils.safe_macro_f1,
    verbose=True)

Accuracy: 0.730
             precision    recall  f1-score   support

        gov      0.726     0.756     0.741      1082
        opp      0.734     0.703     0.718      1039

avg / total      0.730     0.730     0.730      2121



Now bigrams on the modified dataset.

In [23]:
_ = sst.experiment(
    bigrams_phi,
    fit_maxent_classifier,
    train_reader=anon_train_reader, 
    assess_reader=None, 
    train_size=0.7,
    class_func=cas_to_gov,
    score_func=utils.safe_macro_f1,
    verbose=True)

Accuracy: 0.756
             precision    recall  f1-score   support

        gov      0.758     0.750     0.754      1058
    memAffi      0.000     0.000     0.000         1
        opp      0.753     0.763     0.758      1062

avg / total      0.755     0.756     0.756      2121



### Logistic Regression without using sst.experiment

#### Setting up parameters

First, choose the feature function to use.

In [24]:
phi = unigrams_phi

Next, choose the reader used for testing. (None gives a random split.)

In [25]:
assess_reader = None

If we're doing a split, what size should we train on?

In [26]:
train_size = 0.7

Next, choose a function for the classes. (We probably want cas_to_gov.)

In [27]:
class_func = cas_to_gov

Do we want to vectorise?

In [28]:
vectorize = True

#### Building the Classifier

Which classifier are we to use?

In [29]:
classifier = LogisticRegression(fit_intercept=True)

Make it into a training function.

In [30]:
def train_func(X, y):   
    mod = classifier
    mod.fit(X, y)
    return mod

Build the dataset.

In [31]:
train = sst.build_dataset(train_reader, phi, class_func, vectorize=vectorize)

#### Setting up the experiment

First, get the data into standardised variables.

In [32]:
X_train = train['X']
y_train = train['y']
X_assess = None
y_assess = None

If we're not using an assess_reader, do a split on the training data. Otherwise, read in the assessment dataset.

In [33]:
if assess_reader == None:
     X_train, X_assess, y_train, y_assess = train_test_split(
            X_train, y_train, train_size=train_size, test_size=None)
else:
    # Assessment dataset using the training vectorizer:
    assess = sst.build_dataset(
        assess_reader,
        phi,
        class_func,
        vectorizer=train['vectorizer'],
        vectorize=vectorize)
    X_assess, y_assess = assess['X'], assess['y']

#### Experiment
Train the model.

In [34]:
mod = train_func(X_train, y_train)

Test the model.

In [35]:
predictions = mod.predict(X_assess)

Print a report.

In [36]:
print('Accuracy: %0.03f' % sst.accuracy_score(y_assess, predictions))
print(classification_report(y_assess, predictions, digits=3))

Accuracy: 0.748
             precision    recall  f1-score   support

        gov      0.748     0.750     0.749      1065
        opp      0.747     0.745     0.746      1056

avg / total      0.748     0.748     0.748      2121

